In [1]:
import csv


"longest_substring" is a utility method used to find the longest substring of s1 in s2 (assumption: a lemma can never be longer than its unmapped form, so s2 will always be a subset of s1)

**note: `i==0` is an edge case because `s2[:-0] != s2`**

In [197]:
def longest_substring(s1, s2):
    for i in range(len(s2)):
        if i==0:
            if s1.find(s2) == 0:
                return s2
        if s1.find(s2[:-(i+1)]) == 0:
            return s2[:-(i+1)]
    return -1

In [119]:
with open("lemma_mappings.txt") as file:
    items = file.readlines()
items = [item.lstrip('\t').rstrip(',\n').split(':') for item in items if len(item) > 1]

for item in items:
    item[0] = item[0].replace('\'','').lstrip('(').rstrip(')').split(', ')
    item[1] = item[1].replace('\'','')

In [153]:
# test cell - make sure slicing is working as intended
#min_end_len = 2
#print(items[0][1])
#print(items[0][1][-2:])
#print()

#for i in range(len(items[0][1])-(min_end_len-1)):
#    print(items[0][1][-min_end_len-i:]+'/'+items[0][0][1])

Create "rules," which is a dict of lemma rules based on word endings of the training data set. The keys are given in the format [word_ending]/[POS_tag] and each corresponding value is a list containing [lemma mapping, count] where "count" is how many times that particular key maps to that bucket in the training data set

In [155]:
rules = {}
min_end_len = 2
for item in items:
    if item[0][0] == item[1]:
        if len(item[1]) < min_end_len:
            continue
        for i in range(len(item[1])-(min_end_len-1)):
            key = item[1][-min_end_len-i:]+'/'+item[0][1]
            if key not in rules:
                rules[key] = [item[1][-min_end_len-i:], 0]
            else:
                rules[key][1] += 1
        
                

In [156]:
sorted(rules.items(), key=lambda x:x[1][1], reverse=True)

[('ic/a', ['ic', 10]),
 ('al/a', ['al', 5]),
 ('ed/a', ['ed', 5]),
 ('tic/a', ['tic', 4]),
 ('nal/a', ['nal', 3]),
 ('my/n', ['my', 3]),
 ('omy/n', ['omy', 3]),
 ('tomy/n', ['tomy', 3]),
 ('ar/a', ['ar', 3]),
 ('lar/a', ['lar', 3]),
 ('ular/a', ['ular', 3]),
 ('ne/n', ['ne', 2]),
 ('ine/n', ['ine', 2]),
 ('is/n', ['is', 2]),
 ('cular/a', ['cular', 2]),
 ('le/a', ['le', 1]),
 ('ble/a', ['ble', 1]),
 ('able/a', ['able', 1]),
 ('sis/n', ['sis', 1]),
 ('in/n', ['in', 1]),
 ('mic/a', ['mic', 1]),
 ('er/n', ['er', 1]),
 ('ng/a', ['ng', 1]),
 ('ing/a', ['ing', 1]),
 ('on/n', ['on', 1]),
 ('ion/n', ['ion', 1]),
 ('tion/n', ['tion', 1]),
 ('ation/n', ['ation', 1]),
 ('ly/r', ['ly', 1]),
 ('stomy/n', ['stomy', 1]),
 ('ostomy/n', ['ostomy', 1]),
 ('nostomy/n', ['nostomy', 1]),
 ('unostomy/n', ['unostomy', 1]),
 ('junostomy/n', ['junostomy', 1]),
 ('ejunostomy/n', ['ejunostomy', 1]),
 ('jejunostomy/n', ['jejunostomy', 1]),
 ('ojejunostomy/n', ['ojejunostomy', 1]),
 ('ctomy/n', ['ctomy', 1]),
 ('ec